In [ ]:
# default_exp metrics

# Metrics

> This contains metrics that are missing in fastai.

In [ ]:
#export
from tsai.imports import *
from fastai.metrics import * 

In [ ]:
#export
def MatthewsCorrCoef(axis=-1, sample_weight=None):
    "Matthews correlation coefficient for binary classification problems"
    return skm_to_fastai(skm.matthews_corrcoef, axis=axis, sample_weight=sample_weight)

In [ ]:
#hide
create_scripts()
beep()

<IPython.core.display.Javascript object>


Current notebook saved.

Converted 000_utils.ipynb.
Converted 001_data.external.ipynb.
Converted 002_data.core.ipynb.
Converted 003_data.transforms.ipynb.
Converted 005_data.tabular.ipynb.
Converted 006_data.validation.ipynb.
Converted 007_metrics.ipynb.
Converted 008_learner.ipynb.
Converted 010_rocket_functions.ipynb.
Converted 100_layers.ipynb.
Converted 100b_models_utils.ipynb.
Converted 101_ResNet.ipynb.
Converted 102_InceptionTime.ipynb.
Converted index.ipynb.

Checking folder: /Users/nacho/Documents/Machine_Learning/Jupyter_Notebooks/timeseries/tsai
Correct conversion! 😃
Total elapsed time 8 s
22-04-2020 07:23:47
